## CAI - Rebalancing data for March 2023 

## Configuring packages

In [71]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
import datetime
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy


infura_url = decouple.config("AVALANCHE_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

import db_funcs

In [72]:
from pycoingecko import CoinGeckoAPI
key = decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [73]:
import plotly.express as px
import plotly.graph_objects as go

In [74]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables and settings

In [75]:
db_key_date = '2023-03-01'
db_liquidity_table = 'cai_liquidities'
db_benchmark_table = 'cai_benchmark_data'
min_weight = 0.005
max_weight = 0.5
supply_threshold = 0.3
liquidity_consistency = 90 # In days
avax_to_savax_split = 0.6
liveness_threshold = 30*6 # In days

In [76]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [77]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [78]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

# Adding AVAX
coins['avalanche-2'] = {'id': 'avalanche-2','symbol':normalize_symb('avax'), 'name':'Avalanche','addr':'0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7',}


##### Market data

In [79]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

cg_tokens_data = cg_tokens_data.append(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'),ignore_index=False)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_82169/3118279002.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [80]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [81]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])
tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]


In [82]:
# Stable coins to remove from tokens list
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

tokens_market_data =  tokens_market_data[~tokens_market_data.index.isin(stablecoins.index)]
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

tokens_market_data

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,6.373956e+09,18.0,20.210000,4.205317e+08,3.151257e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
uniswap,Uniswap,UNI,5.366943e+09,19.0,7.120000,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
wrapped-bitcoin,Wrapped Bitcoin,WBTC,4.328085e+09,21.0,24625.000000,1.754971e+05,1.754971e+05,0x50b7545627a5162f82a992c33b87adc75187b218
chainlink,Chainlink,LINK,3.826398e+09,23.0,7.780000,1.000000e+09,4.916000e+08,0x5947bb275c521040051d82396192181b413227a3
the-graph,The Graph,GRT,1.563911e+09,41.0,0.175454,1.000000e+10,8.893377e+09,0x8a0cac13c7da965a312f08ea4229c37869e85cb9
...,...,...,...,...,...,...,...,...
fortressdao,Fortress,FORT,1.196430e+04,3728.0,0.008534,1.409126e+06,1.401945e+06,0xf6d46849db378ae01d93732585bec2c4480d1fd5
morphie,Morphie,MRFI,8.163660e+03,3764.0,0.000001,2.000000e+10,8.069296e+09,0x05e481b19129b560e921e487adb281e70bdba463
hermes,HERMES,HERMES,2.930000e+03,3852.0,0.478798,9.878883e+03,6.119485e+03,0xb15f02f9da8cd1f99e9dd375f21dc96d25ddd82c


### Token's supply check 

In [83]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > supply_threshold
sp_passed = tokens_market_data[sp_passed_index]

### Retrieve historical data & token age check

In [84]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in sp_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < liveness_threshold:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {liveness_threshold} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < liveness_threshold:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {liveness_threshold} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Note: BTCB, marketcap data available only for 127 < 180 days
Excluding USP, prices data available only for 12 < 180 days
Excluding XEN, prices data available only for 134 < 180 days
Note: CLY, marketcap data available only for 172 < 180 days
Note: BUILD, marketcap data available only for 179 < 180 days
Note: KENSHI, marketcap data available only for 19 < 180 days
Note: 3ULL, marketcap data available only for 151 < 180 days
Note: DCAR, marketcap data available only for 160 < 180 days
Excluding CAI, prices data available only for 168 < 180 days
Note: MAGIC, marketcap data available only for 41 < 180 days
Note: CHRO, marketcap data available only for 106 < 180 days
Note: FRF, marketcap data available only for 14 < 180 days


### Non quantative checks

In [85]:
manual_exclusions = [
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'wrapped-bitcoin',
        'reason' : 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'bitcoin-avalanche-bridged-btc-b',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'woo-network',
        'reason': 'Bridged asset'
    }
]
for token in manual_exclusions:
    exclude_list.append(token['gecko_id'])

In [109]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list')
passed_q_and_non_q_checks

name                                                       BENQI
symbol                                                        QI
market_cap                                            39469333.0
market_cap_rank                                            562.0
current_price                                           0.011435
total_supply                                        7200000000.0
circulating_supply                                  3448848275.0
addr                  0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
Name: benqi, dtype: object

### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [87]:
url_0x = "https://avalanche.api.0x.org/swap/v1/quote"

In [88]:
decimals = 18
slippages = []
for id, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': data['addr'],
            'sellToken': 'AVAX',
            'sellAmount': int(trade_value_tiny / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals)
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'id': id,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        continue
    

In [ ]:
slippage

In [108]:
slippages_df = pd.DataFrame(slippages).sort_values('slippage',ascending=False).reset_index(drop=True)
slippages_df

,id,spot_price,delivery_price,slippage
0,benqi-liquid-staked-avax,9.397780e-01,9.399232e-01,0.000154
1,avalanche-2,1.000000e+00,1.000000e+00,0.000000
2,chainlink,2.587019e+00,2.188480e+00,-0.154053
3,aave,2.265191e-01,1.508999e-01,-0.333831
4,alpha-finance,1.315670e+02,8.709456e+01,-0.338021
5,kyber-network-crystal,2.313634e+01,1.038062e+01,-0.551328
6,synapse-2,1.513930e+01,2.570705e+00,-0.830197
7,ankr,5.900462e+02,9.417302e+01,-0.840397
8,ape-in,5.962964e+01,6.576449e+00,-0.889712
9,fortressdao,2.344100e+03,1.966139e+02,-0.916124


In [90]:
# Writing to database
slippages_to_save = slippages_df[['id','slippage']].head(20)
asset_columns = db_funcs.convert_to_sql_strings(list(slippages_to_save['id']))
slippage_values = list(slippages_to_save['slippage'])
db_funcs.insert_values(db_key_date,asset_columns,slippage_values,db_liquidity_table)


In [91]:
def check_avg_liquidity(date_delta):
    start_date = str(datetime.date.fromisoformat(db_key_date) - datetime.timedelta(days=date_delta))
    dataframe = pd.read_sql(f'Select * from {db_liquidity_table} where date >= ? ',db_funcs.create_connection(db_funcs.db),index_col='date',parse_dates=['date'],params=[start_date])
    dataframe = dataframe.mean(axis=0)
    dataframe = dataframe[dataframe > max_slippage]
    dataframe.index = db_funcs.convert_from_sql_strings(list(dataframe.index))
    return dataframe, list(dataframe.index)

avg_liq_dataframe ,include_list = check_avg_liquidity(liquidity_consistency)


In [92]:
include_list = list(slippages_df[abs(slippages_df['slippage']) < abs(max_slippage)]['id'])
qualified_assets = passed_q_and_non_q_checks.query('index in @include_list')
qualified_assets


,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,6.373956e+09,18.0,20.21,4.205317e+08,3.151257e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.139832e+08,288.0,21.58,5.284799e+06,5.284799e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


### Marketcap ranking & filtering

In [93]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [94]:
# exclude savax from weighting since it should be a part of avax weight
if 'benqi-liquid-staked-avax' in chosen_tokens.index:
    chosen_tokens_ex_savax = chosen_tokens.drop('benqi-liquid-staked-avax')
    weights = chosen_tokens_ex_savax['market_cap'].div(chosen_tokens_ex_savax['market_cap'].sum())
else:
    weights = chosen_tokens['market_cap'].div(chosen_tokens['market_cap'].sum())



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [95]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum()
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum()).mul(c)
    
    return w

In [96]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum())
    
    return w

In [97]:
adjusted_weights = adjust_weights(weights, max_weight=max_weight) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=min_weight)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=max_weight)

### Split AVAX 

In [98]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * avax_to_savax_split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - avax_to_savax_split)

adjusted_weights =  adjusted_weights.sort_values(ascending = False)


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [99]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [100]:
converted_weights = convert_weights(adjusted_weights)

## Summary table

In [101]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = chosen_tokens['market_cap']
cai['price'] = chosen_tokens['current_price']
cai['weight'] = adjusted_weights
cai['weight_converted'] = converted_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,6.373956e+09,20.21,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.139832e+08,21.58,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


In [102]:
# Add assets and weights to database
db_assets = db_funcs.convert_to_sql_strings(list(cai.index))
db_weights = list(cai['weight'])
db_funcs.insert_values(db_key_date,db_assets,db_weights,db_benchmark_table)

In [103]:
#Querying current contract to ascertain which assets will be dropped

cai_token_address = '0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0'
ia_contract = web3_object.eth.contract(address=cai_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()


for address in addresses:
    address = address.lower()
    if address not in list(cai['address']):
        data = tokens_market_data[tokens_market_data['addr']== address]
        cai.loc[data.index[0]] = [data['name'][0],data['symbol'][0],data['market_cap'][0],data['current_price'][0],0,0,address]

cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,6.373956e+09,20.210000,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.139832e+08,21.580000,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,9.838522e+07,0.294152,0.0,0,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,3.946933e+07,0.011435,0.0,0,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5


In [104]:
zipped_assets = list(zip(list(cai['address']),list(cai['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
[76, 0, 0, 51]
